### Open three files:
    -company_list.pickle(mapping: organization ---> company)
    -news data(from GDELT)
    -theme_to_SDG(dictionary which shows correspnding SDG type for each theme)

In [24]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from datetime import timedelta

In [30]:
# We need to change this in big computer so that we can use news data(2015 to 2019)
path = 'E:\DataGKG2\GKG2HistoricalData'
os.chdir(path)
ready_file = os.listdir(path)
ready_file = [i for i in ready_file if 'pkl' in i]
ready_file

['20180102.pkl',
 '20180103.pkl',
 '20180104.pkl',
 '20180105.pkl',
 '20180106.pkl',
 '20180107.pkl',
 '20180108.pkl',
 '20180109.pkl',
 '20180110.pkl',
 '20180111.pkl',
 '20180112.pkl',
 '20180113.pkl',
 '20180114.pkl',
 '20180115.pkl',
 '20180116.pkl',
 '20180117.pkl',
 '20180118.pkl',
 '20180119.pkl',
 '20180120.pkl',
 '20180121.pkl',
 '20180122.pkl',
 '20180123.pkl',
 '20180124.pkl',
 '20180125.pkl',
 '20180126.pkl',
 '20180127.pkl',
 '20180128.pkl',
 '20180129.pkl',
 '20180130.pkl',
 '20180131.pkl',
 '20180210.pkl',
 '20180211.pkl',
 '20180212.pkl',
 '20180213.pkl',
 '20180214.pkl',
 '20180215.pkl',
 '20180216.pkl',
 '20180217.pkl',
 '20180218.pkl',
 '20180219.pkl',
 '20180220.pkl',
 '20180221.pkl',
 '20180222.pkl',
 '20180223.pkl',
 '20180224.pkl',
 '20180225.pkl',
 '20180226.pkl',
 '20180227.pkl',
 '20180228.pkl',
 '20180301.pkl',
 '20180302.pkl',
 '20180303.pkl',
 '20180304.pkl',
 '20180305.pkl',
 '20180306.pkl',
 '20180307.pkl',
 '20180308.pkl',
 '20180309.pkl',
 '20180310.pkl

In [31]:
to_do_file = [i for i in date_list if i not in ready_file]
to_do_file

['20180101.pkl']

In [3]:
# read company_list into dictionary
df = pd.read_pickle('company_list.pickle')
company_dict = {}

for key,value in df.items():
    company_dict[key.lower()] = value.lower()
company_key = list(company_dict.keys())

In [6]:
# read theme_to_SDG
df_theme = pd.read_pickle('theme_sdg_mapping.pk')

#read news data
df_news = pd.read_pickle('20190101.pkl')

# return columns we are interested in
news = df_news[['DATE','DocumentIdentifier','V2Tone','Themes','Organizations']]
news.head()

,DATE,DocumentIdentifier,V2Tone,Themes,Organizations
0,2.019010e+13,http://www.buzznet.com/2018/12/inside-the-live...,"0.444444444444444,3.11111111111111,2.666666666...",TAX_ECON_PRICE;TAX_FNCACT;TAX_FNCACT_ACTRESS;T...,united kingdom press pool united kingdom;eden ...
1,2.019010e+13,http://gwdtoday.com/main.asp?SectionID=2&SubSe...,"2.73081924577373,3.94451668833984,1.2136974425...",MANMADE_DISASTER_IMPLIED;EDUCATION;SOC_POINTSO...,maggie hamm- library club;james medford family...
2,2.019010e+13,https://www.lamonitor.com/content/new-mexico-p...,"0.72463768115942,3.6231884057971,2.89855072463...",LEADER;TAX_FNCACT;TAX_FNCACT_GOVERNOR;TAKE_OFF...,santa fe convention center
3,2.019010e+13,https://www.nbc15.com/content/news/Winter-weat...,"-2.18181818181818,0.727272727272727,2.90909090...",CRISISLEX_T01_CAUTION_ADVICE;MANMADE_DISASTER_...,dane county sheriff office;sheriff office depa...
4,2.019010e+13,https://magicvalley.com/entertainment/the-best...,"3.97003745318352,8.68913857677903,4.7191011235...",EPU_ECONOMY_HISTORIC;TAX_FNCACT;TAX_FNCACT_COR...,writers guild;lionel hahn abaca press;hollywoo...


In [ ]:
for file_name in ready_file:
    news = pd.read_pickle(file_name)[['DATE','DocumentIdentifier','V2Tone','Themes','Organizations']]
    news_clean = news[~news['Organizations'].isna()]
    news_clean = news_clean[~news_clean['Themes'].isna()]
    news_clean = news_clean.drop_duplicates(subset=  'V2Tone',keep='first')
    
    index = news_clean['Organizations'].apply(check_row_in_company_key_new)
    news_clean['Organizations'] = index
    news_3 =  news_clean.loc[index[~index.isnull()].index]
    
    # build a dictionary cc used for being transformed into datafrmae
    # you can view cc after running this code
    news_4 = news_3.copy()
    news_4['V2Tone'] = news_3['V2Tone'].apply(return_first_v2tone_score)
    theme_list = [x.split(';') for x in list(news_4['Themes'].values)]
    v2tone_list = list(news_4['V2Tone'].values)
    company_news_count = {}
    company_to_url = {}
    sdg_count = {}
    company_sdg_count = {}

    cc = {}
    ii = 0

    for l in news_4['Organizations'].iloc[0:news_4.shape[0]]:
        # we need to initialize sdg_init (these are for other company)
        sdg_init = {}

        for i in range(1,18):
            sdg_init['SDG_'+str(i)] =0 
            sdg_count['SDG_'+str(i)+'_count'] = 0

        theme = theme_list[ii]
        score = v2tone_list[ii]

        for i in theme:

            if i in df_theme:
                for category in df_theme[i]:
                    if sdg_init['SDG_'+category] ==0:
                        sdg_init['SDG_'+category] = score

            else:
                continue

        # add sdg_count to company_sdg_count 


        #print('sdg_init is ', sdg_init)

        for company in l:
            value_sdg_init = list(sdg_init.values())
            value_sdg_count= list(sdg_count.values())

            if company in company_sdg_count:
                value_company = list(company_sdg_count[company].values())
                value_new = [y if x ==0  else y+1  for x,y in zip(value_sdg_init,value_company)]
                company_sdg_count[company] = dict(zip(sdg_count.keys(),value_new))            

            else:
                value_new = [y if x ==0  else y+1  for x,y in zip(value_sdg_init,value_sdg_count)]
                company_sdg_count[company] = dict(zip(sdg_count.keys(),value_new))        

            if company in company_to_url:
                company_to_url[company].append(news_4['DocumentIdentifier'].iloc[ii])

            else:
                company_to_url[company] = [news_4['DocumentIdentifier'].iloc[ii]]

            if company in cc:
                value_1 = [x for x in sdg_init.values()]
                value_2 = [x for x in cc[company].values()]

                #computing average for every new sample.
                value = [(x*company_news_count[company]+y)/(company_news_count[company]+1) for x,y in zip(value_2,value_1)]
                cc[company] = dict(zip(sdg_init.keys(),value))
                #print('num is ',company_news_count[company])
                #print('berkshire hathaway' ,cc['berkshire hathaway'])
                #print('value 2 is ',value_2)
                company_news_count[company] +=1

            else:
                cc[company] = sdg_init
                company_news_count[company] = 1
                #print('first time')

        ii+=1
    dd = pd.DataFrame.from_dict(cc)
    data = dd.T
    data['url'] = data.index
    data['url'] = data['url'].map(company_to_url)
    data.head()
    df_sdg_count = pd.DataFrame.from_dict(company_sdg_count).T
    data_new = pd.concat([data, df_sdg_count],axis=1)
    date = [file_name[:-4]]*len(data_new)
    data_new.insert(0, 'date', date)
    data_new.to_csv(file_name[:-4]+'.csv')

In [7]:
news.shape

(120766, 5)

In [8]:
# check whether there exists np.nan in some columns
print(news['Organizations'].isnull().any())
print(news['Themes'].isnull().any())

True
True


In [9]:
#drop missing value for org and themes; drop duplicate value for V2Tone(there are duplicate news which can affect our research performance)
news_clean = news[~news['Organizations'].isna()]
news_clean = news_clean[~news_clean['Themes'].isna()]
news_clean = news_clean.drop_duplicates(subset=  'V2Tone',keep='first')

In [10]:
# check how many news is left, remember, we still haven't returned relevant companies news.
news_clean.shape

(45251, 5)

In [11]:
# return news for relevant companies
def check_row_in_company_key_new(x):
    l = x.split(';')
    company_related = []
    
    for i in l :
        if i in company_key:
            company_related.append(i)
            
        else:
            continue
    
    if len(company_related)==0:
        return np.nan
    
    else:
        return company_related

In [12]:
#notice we need to transform V2Tone into scaler after returning relevant company news
index = news_clean['Organizations'].apply(check_row_in_company_key_new)
news_clean['Organizations'] = index
news_3 =  news_clean.loc[index[~index.isnull()].index]

news_3['V2Tone'].head()

6     -2.2508038585209,1.82207931404073,4.0728831725...
20    -0.0980392156862742,3.82352941176471,3.9215686...
25    0.425531914893617,1.27659574468085,0.851063829...
46    1.75438596491228,1.75438596491228,0,1.75438596...
51    1.27499275572298,2.37612286293828,1.1011301072...
Name: V2Tone, dtype: object

In [13]:
# return the first element in each V2Tone vector(you need to check document to verify the meaning of other elements)
def return_first_v2tone_score(x):
    l = x.split(',')
    l = [float(x) for x in l]
    return l[0]


news_4 = news_3.copy()
news_4['V2Tone'] = news_3['V2Tone'].apply(return_first_v2tone_score)
news_4.head()

,DATE,DocumentIdentifier,V2Tone,Themes,Organizations
6,2.019010e+13,https://www.wbaltv.com/article/wall-street-end...,-2.250804,ECON_STOCKMARKET;CRISISLEX_CRISISLEXREC;MANMAD...,"[google, nasdaq]"
20,2.019010e+13,https://www.cairnspost.com.au/entertainment/mo...,-0.098039,TAX_FNCACT;TAX_FNCACT_AUTHOR;TAX_ETHNICITY;TAX...,[walt disney]
25,2.019010e+13,http://smokeroom.com/2018/12/31/caitlin-arnett...,0.425532,TAX_FNCACT;TAX_FNCACT_WOMEN;WB_678_DIGITAL_GOV...,[instagram]
46,2.019010e+13,https://www.newsoptimist.ca/most-actively-trad...,1.754386,WB_698_TRADE;ECON_STOCKMARKET;WB_2936_GOLD;WB_...,"[aurora cannabis inc, canadian pacific railway..."
51,2.019010e+13,https://www.funkytaurusmedia.com/shop_content....,1.274993,WB_678_DIGITAL_GOVERNMENT;WB_694_BROADCAST_AND...,"[google, instagram, linkedin]"


In [14]:
# Now we check how much news is left
news_4.shape

(4848, 5)

In [175]:
# no actural meaning in this part. Just for testing.

total = 0
num = 0
for i in range(200):
    if 'berkshire hathaway' in news_4['Organizations'].iloc[i] :
        
        for theme in theme_list[i]:
            if theme in df_theme:
                print(news_4['Organizations'].iloc[i])
                print(news_4['V2Tone'].iloc[i])
                
                
                
                print((total + news_4['V2Tone'].iloc[i])/(num+1))
                total = total + news_4['V2Tone'].iloc[i]
                
                num +=1
                
                break
                
            else:
                continue
                
print(total/num)

['berkshire hathaway']
0.99601593625498
0.99601593625498
['garmin ltd', 'nasdaq', 'exxon', 'berkshire hathaway']
-0.727272727272727
0.1343716044911265
['exxon', 'berkshire hathaway']
0.533807829181494
0.267517012721249
0.267517012721249


In [ ]:
# I want to build a dictionary like the following one so that it can be easily and quickly transformed into dataframe
cc = {company_1: {SDG_1:0,SDG_2:0,SDG_3:0,SDG_4:0,SDG_5:0},company_2: {SDG_1:0,SDG_2:0,SDG_3:0,SDG_4:0,SDG_5:0} }

In [16]:
# I build two list for "theme" and "v2tone" columns
theme_list = [x.split(';') for x in list(news_4['Themes'].values)]
v2tone_list = list(news_4['V2Tone'].values)

In [17]:
# build a dictionary cc used for being transformed into datafrmae
# you can view cc after running this code

company_news_count = {}
company_to_url = {}
sdg_count = {}
company_sdg_count = {}

cc = {}
ii = 0


for l in news_4['Organizations'].iloc[0:news_4.shape[0]]:
    # we need to initialize sdg_init (these are for other company)
    sdg_init = {}

    for i in range(1,18):
        sdg_init['SDG_'+str(i)] =0 
        sdg_count['SDG_'+str(i)+'_count'] = 0

    theme = theme_list[ii]
    score = v2tone_list[ii]

    for i in theme:

        if i in df_theme:
            for category in df_theme[i]:
                if sdg_init['SDG_'+category] ==0:
                    sdg_init['SDG_'+category] = score

        else:
            continue
            
    # add sdg_count to company_sdg_count 

    
    #print('sdg_init is ', sdg_init)

    for company in l:
        value_sdg_init = list(sdg_init.values())
        value_sdg_count= list(sdg_count.values())
        
        
        if company in company_sdg_count:
            value_company = list(company_sdg_count[company].values())
            value_new = [y if x ==0  else y+1  for x,y in zip(value_sdg_init,value_company)]
            company_sdg_count[company] = dict(zip(sdg_count.keys(),value_new))            
            
        else:
            value_new = [y if x ==0  else y+1  for x,y in zip(value_sdg_init,value_sdg_count)]
            company_sdg_count[company] = dict(zip(sdg_count.keys(),value_new))        
          
        
        if company in company_to_url:
            company_to_url[company].append(news_4['DocumentIdentifier'].iloc[ii])
        
        else:
            company_to_url[company] = [news_4['DocumentIdentifier'].iloc[ii]]
        
        if company in cc:
            value_1 = [x for x in sdg_init.values()]
            value_2 = [x for x in cc[company].values()]

            #computing average for every new sample.
            value = [(x*company_news_count[company]+y)/(company_news_count[company]+1) for x,y in zip(value_2,value_1)]
            cc[company] = dict(zip(sdg_init.keys(),value))
            #print('num is ',company_news_count[company])
            #print('berkshire hathaway' ,cc['berkshire hathaway'])
            #print('value 2 is ',value_2)
            company_news_count[company] +=1

        else:
            cc[company] = sdg_init
            company_news_count[company] = 1
            #print('first time')

    ii+=1


In [18]:
cc

{'google': {'SDG_1': -0.06575830520286434,
  'SDG_2': 0.00847714461272325,
  'SDG_3': -0.4853585208790772,
  'SDG_4': -0.029256519394995566,
  'SDG_5': -0.2987591855884958,
  'SDG_6': -0.1401275514103107,
  'SDG_7': -0.2771349464889373,
  'SDG_8': -0.6190801422068205,
  'SDG_9': -0.46336764564424887,
  'SDG_10': -0.6687785333028807,
  'SDG_11': -0.7918398788357254,
  'SDG_12': -0.2678423849000751,
  'SDG_13': -0.11692322258956164,
  'SDG_14': -0.03165902258407603,
  'SDG_15': -0.11484169114507653,
  'SDG_16': -0.9327822909241696,
  'SDG_17': -0.1183160598450301},
 'nasdaq': {'SDG_1': -0.02326666122779756,
  'SDG_2': 0.012590243960528174,
  'SDG_3': -0.3048760512001555,
  'SDG_4': 0.02174921970934815,
  'SDG_5': -0.02363749402213372,
  'SDG_6': -0.13591139314869288,
  'SDG_7': -0.5209936600268242,
  'SDG_8': -0.5536159612437607,
  'SDG_9': -0.16690556929928788,
  'SDG_10': -0.7284319017879543,
  'SDG_11': -0.2616452919028257,
  'SDG_12': -0.14806037258669327,
  'SDG_13': -0.071093140199

In [19]:
dd = pd.DataFrame.from_dict(cc)
data = dd.T

In [20]:
data['url'] = data.index
data['url'] = data['url'].map(company_to_url)
data.head()

,SDG_1,SDG_10,SDG_11,SDG_12,SDG_13,SDG_14,SDG_15,SDG_16,SDG_17,SDG_2,SDG_3,SDG_4,SDG_5,SDG_6,SDG_7,SDG_8,SDG_9,url
google,-0.065758,-0.668779,-0.791840,-0.267842,-0.116923,-0.031659,-0.114842,-0.932782,-0.118316,0.008477,-0.485359,-0.029257,-0.298759,-0.140128,-0.277135,-0.619080,-0.463368,[https://www.wbaltv.com/article/wall-street-en...
nasdaq,-0.023267,-0.728432,-0.261645,-0.148060,-0.071093,0.009159,-0.072656,-0.578786,-0.296750,0.012590,-0.304876,0.021749,-0.023637,-0.135911,-0.520994,-0.553616,-0.166906,[https://www.wbaltv.com/article/wall-street-en...
walt disney,-0.060843,0.558475,0.513547,0.116562,-0.052535,0.000000,-0.052535,0.261007,0.000000,0.168813,0.334754,0.377771,0.000000,0.021155,0.021155,0.211902,0.176533,[https://www.cairnspost.com.au/entertainment/m...
instagram,0.040701,0.600997,0.264764,0.028901,0.065744,0.039205,0.066784,0.029313,0.014710,0.013540,0.540460,0.088116,-0.013048,0.010522,0.008177,0.158874,0.036664,[http://smokeroom.com/2018/12/31/caitlin-arnet...
aurora cannabis inc,0.584795,0.824259,2.042897,0.000000,0.000000,0.000000,0.000000,0.239464,0.000000,0.584795,2.042897,0.000000,0.000000,0.000000,1.458102,2.042897,0.000000,[https://www.newsoptimist.ca/most-actively-tra...


In [21]:
data.shape

(601, 18)

In [22]:
df_sdg_count = pd.DataFrame.from_dict(company_sdg_count).T
data_new = pd.concat([data, df_sdg_count],axis=1)
data_new

,SDG_1,SDG_10,SDG_11,SDG_12,SDG_13,SDG_14,SDG_15,SDG_16,SDG_17,SDG_2,...,SDG_17_count,SDG_1_count,SDG_2_count,SDG_3_count,SDG_4_count,SDG_5_count,SDG_6_count,SDG_7_count,SDG_8_count,SDG_9_count
google,-0.065758,-0.668779,-0.791840,-0.267842,-0.116923,-0.031659,-0.114842,-0.932782,-0.118316,0.008477,...,42,47,40,289,161,67,41,76,319,216
nasdaq,-0.023267,-0.728432,-0.261645,-0.148060,-0.071093,0.009159,-0.072656,-0.578786,-0.296750,0.012590,...,75,24,36,307,60,5,64,156,491,126
walt disney,-0.060843,0.558475,0.513547,0.116562,-0.052535,0.000000,-0.052535,0.261007,0.000000,0.168813,...,0,2,4,11,7,0,1,1,17,9
instagram,0.040701,0.600997,0.264764,0.028901,0.065744,0.039205,0.066784,0.029313,0.014710,0.013540,...,45,88,68,750,269,71,14,31,354,319
aurora cannabis inc,0.584795,0.824259,2.042897,0.000000,0.000000,0.000000,0.000000,0.239464,0.000000,0.584795,...,0,1,1,3,0,0,0,2,3,0
canadian pacific railway ltd,-0.352079,-0.352079,-0.352079,0.000000,-0.397592,-0.083205,-0.397592,-0.488172,0.000000,-0.352079,...,0,6,6,2,0,0,0,1,2,0
barrick gold corp,0.877193,0.071741,0.071741,-0.805452,0.000000,0.000000,0.000000,-0.805452,-0.805452,0.877193,...,1,1,1,1,0,0,1,1,2,0
canadian national railway co,-0.307284,-0.307284,-0.307284,0.000000,-0.454391,0.000000,-0.454391,-0.557911,0.000000,-0.307284,...,0,5,5,1,0,0,0,0,1,0
bombardier inc,1.754386,1.754386,1.754386,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.754386,...,0,1,1,1,0,0,0,0,1,0
linkedin,0.047387,-0.130230,0.074953,0.356470,-0.010608,0.016467,-0.028574,-0.103975,0.092938,0.004850,...,7,4,14,31,31,8,2,4,50,24


In [87]:
company_dict

{'microsoft corp': 'microsoft corp',
 'apple inc': 'apple inc',
 'braeburn capital': 'apple inc',
 'beats electronics': 'apple inc',
 'filemaker inc.': 'apple inc',
 'anobit': 'apple inc',
 'beddit': 'apple inc',
 'shazam (application)': 'apple inc',
 'amazon.com inc': 'amazon.com inc',
 'a9.com': 'amazon.com inc',
 'abebooks': 'amazon.com inc',
 'amazon air': 'amazon.com inc',
 'alexa internet': 'amazon.com inc',
 'amazon books': 'amazon.com inc',
 'amazon game studios': 'amazon.com inc',
 'amazon lab126': 'amazon.com inc',
 'amazon publishing': 'amazon.com inc',
 'amazon robotics': 'amazon.com inc',
 'amazon studios': 'amazon.com inc',
 'audible inc.': 'amazon.com inc',
 'body labs': 'amazon.com inc',
 'amazon web services': 'amazon.com inc',
 'book depository': 'amazon.com inc',
 'comixology': 'amazon.com inc',
 'goodreads': 'amazon.com inc',
 'graphiq': 'amazon.com inc',
 'imdb': 'amazon.com inc',
 'ring (company)': 'amazon.com inc',
 'souq.com': 'amazon.com inc',
 'twitch.tv': 'am

In [26]:
data_new

,SDG_1,SDG_10,SDG_11,SDG_12,SDG_13,SDG_14,SDG_15,SDG_16,SDG_17,SDG_2,...,SDG_17_count,SDG_1_count,SDG_2_count,SDG_3_count,SDG_4_count,SDG_5_count,SDG_6_count,SDG_7_count,SDG_8_count,SDG_9_count
google,0.078687,-0.005987,-0.029302,0.050211,-0.012710,0.005130,-0.016510,-0.254390,-0.000559,0.110813,...,63,171,174,545,391,70,113,49,561,469
nasdaq,-0.041348,-0.174739,-0.025998,-0.093399,-0.119697,0.012462,-0.122084,-0.146830,-0.106744,-0.015763,...,155,58,44,475,195,3,41,262,850,272
instagram,0.151785,0.683635,0.240061,0.053961,0.104134,0.068642,0.095144,-0.093306,-0.002691,0.172117,...,26,129,147,901,396,221,42,48,479,512
barclays,0.016589,-0.161748,-0.543669,-0.069865,-0.020049,-0.008248,-0.020049,-0.592917,-0.033503,0.107102,...,8,6,8,85,18,1,11,17,102,30
linkedin,0.058920,0.430166,0.834638,0.183396,0.074718,0.035925,0.074718,0.306936,0.090059,0.072606,...,30,16,23,171,166,18,11,26,194,121
ventas inc,0.000000,0.000000,1.552795,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,1,1,0,0,0,1,0
keycorp,0.033798,0.179393,0.902027,0.047259,0.000000,0.000000,0.000000,0.520178,0.000000,0.000000,...,0,1,0,17,3,0,0,0,18,4
hutchison port holdings,0.000000,1.189326,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,2,2,0
singtel,0.000000,0.475730,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,2,5,0
samsung,0.151406,0.301800,0.361408,0.032070,-0.004857,0.003266,-0.004857,0.211045,0.139057,0.159472,...,47,108,110,185,120,6,90,10,218,227


In [27]:
date = ['20190101']*len(data_new)
data_new.insert(0, 'date', date)
data_new

,date,SDG_1,SDG_10,SDG_11,SDG_12,SDG_13,SDG_14,SDG_15,SDG_16,SDG_17,...,SDG_17_count,SDG_1_count,SDG_2_count,SDG_3_count,SDG_4_count,SDG_5_count,SDG_6_count,SDG_7_count,SDG_8_count,SDG_9_count
google,20190101,0.078687,-0.005987,-0.029302,0.050211,-0.012710,0.005130,-0.016510,-0.254390,-0.000559,...,63,171,174,545,391,70,113,49,561,469
nasdaq,20190101,-0.041348,-0.174739,-0.025998,-0.093399,-0.119697,0.012462,-0.122084,-0.146830,-0.106744,...,155,58,44,475,195,3,41,262,850,272
instagram,20190101,0.151785,0.683635,0.240061,0.053961,0.104134,0.068642,0.095144,-0.093306,-0.002691,...,26,129,147,901,396,221,42,48,479,512
barclays,20190101,0.016589,-0.161748,-0.543669,-0.069865,-0.020049,-0.008248,-0.020049,-0.592917,-0.033503,...,8,6,8,85,18,1,11,17,102,30
linkedin,20190101,0.058920,0.430166,0.834638,0.183396,0.074718,0.035925,0.074718,0.306936,0.090059,...,30,16,23,171,166,18,11,26,194,121
ventas inc,20190101,0.000000,0.000000,1.552795,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,1,1,0,0,0,1,0
keycorp,20190101,0.033798,0.179393,0.902027,0.047259,0.000000,0.000000,0.000000,0.520178,0.000000,...,0,1,0,17,3,0,0,0,18,4
hutchison port holdings,20190101,0.000000,1.189326,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,2,2,0
singtel,20190101,0.000000,0.475730,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,2,5,0
samsung,20190101,0.151406,0.301800,0.361408,0.032070,-0.004857,0.003266,-0.004857,0.211045,0.139057,...,47,108,110,185,120,6,90,10,218,227
